# Using ESMFold

This tutorial shows you how to use the ESMFold model to create a PDB of your protein sequence of interest. We recommend using ESMFold with single-chain sequences. If you have a multi-chain sequence, please visit [Using AlphaFold2](https://colab.research.google.com/drive/1hDACQWTuxfE_-WFVF4ivyPPebUizV-EA#scrollTo=AzVrAlxU4daB).

## What you need before getting started

Specify a sequence of interest whose structure you want to predict. The example used here is interleukin 2:

In [ ]:
sequence = "MYRMQLLSCIALSLALVTNSAPTSSSTKKTQLQLEHLLLDLQMILNGINNYKNPKLTRMLTFKFYMPKKATELKHLQCLEEELKPLEEVLNLAQSKNFHLRPRDLISNINVIVLELKGMYRMQLLSCIALSLALVTNSAPTSSSTKKTQLQLEHLLLDLQMILNGINNYKNPKLTRMLTFKFYMPKKATELKHLQCLEEELKPLEEVLNLAQSKNFHLRPRDLISNINVIVLELKGSEP"


## Predicting your sequence

Call ESMFold on your sequence. The `num_recycles` hyperparameter allows the model to further refine structures using the previous cycle’s output as the new cycle’s input. This parameter accepts integers between 1 and 48.


Create the model object for ESMFold:

In [ ]:
esmfoldmodel = session.fold.get_model('esmfold')
esmfoldmodel.fold?

Send the sequence of interest to ESM for folding:

In [ ]:
esm = esmfoldmodel.fold([sequence.encode()], num_recycles=1)

esm

Wait for the job to complete with `wait_until_done()`:


In [ ]:
esm.wait_until_done(verbose=True, timeout=300)

Waiting: 100%|██████████| 100/100 [00:00<00:00, 1345.39it/s, status=SUCCESS]


True

Fetch the results with `get()`




The results display a tuple containing the query sequence and the contents of the resulting PDB file:


In [ ]:
result = esm.get()
print(result[0][0])

b'MYRMQLLSCIALSLALVTNSAPTSSSTKKTQLQLEHLLLDLQMILNGINNYKNPKLTRMLTFKFYMPKKATELKHLQCLEEELKPLEEVLNLAQSKNFHLRPRDLISNINVIVLELKGMYRMQLLSCIALSLALVTNSAPTSSSTKKTQLQLEHLLLDLQMILNGINNYKNPKLTRMLTFKFYMPKKATELKHLQCLEEELKPLEEVLNLAQSKNFHLRPRDLISNINVIVLELKGSEP'


Return a PDB file:

In [ ]:
print("\n".join( list(result[0][1].decode().split("\n")[0:5]) ) )

PARENT N/A
ATOM      1  N   MET A   1       0.462 -12.684 -24.850  1.00 49.54           N  
ATOM      2  CA  MET A   1       1.650 -11.836 -24.842  1.00 51.45           C  
ATOM      3  C   MET A   1       1.718 -11.007 -23.563  1.00 49.89           C  
ATOM      4  CB  MET A   1       1.663 -10.916 -26.063  1.00 46.31           C  


## Next steps

After the PDB contents are returned, save them as a file for use with your molecular visualization system of choice.